In [ ]:
!pip install langchain
!pip install openai
!pip install datasets
!pip install evaluate

In [ ]:
from datasets import load_dataset
dataset = load_dataset('akufeldt/fr-gec-dataset')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/59850 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3325 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/3325 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/akufeldt___parquet/akufeldt--fr-gec-dataset-dc2539dd430be97d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Zero Shot Prompting with GPT3.5

In [ ]:
from langchain.chat_models import ChatOpenAI

# initialize the models
gpt3 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="YOUR_API_KEY"
)

In [ ]:
# zero shot
from langchain import PromptTemplate

gecprompt = PromptTemplate(
    input_variables=['sent'],
    template='Correct the errors in the following French text without telling me what you did: {sent} \n',
)

In [ ]:
from langchain.chains import LLMChain
gecchain = LLMChain(llm=gpt3, prompt=gecprompt)

# Few Shot Prompting with GPT3.5

In [ ]:
# few shot

examples = []
for i in range(51,54):
  inp = dataset['test']['modified'][i][13:]
  ans = dataset['test']['sentence'][i]
  thing = 'Correct the errors in the following French text without telling me what you did: ' + inp + '\n'+ans
  examples.append(thing)

print(examples)

from langchain import PromptTemplate

gecprompt = PromptTemplate(
    input_variables=['sent'],
    template='\n\n'.join(examples)+'Correct the errors in the following French text without telling me what you did: {sent} \n',
)

["Correct the errors in the following French text without telling me what you did: Il offrait également le option de deux portes latérales coulissantes ainsi que deux portes ou un hayon à le arrière, alors que aucun de ses principaux concurrents ne a cette option.\nIl offrait également l'option de deux portes latérales coulissantes ainsi que deux portes ou un hayon à l'arrière, alors qu'aucun de ses principaux concurrents n'a cette option.", 'Correct the errors in the following French text without telling me what you did: La HIAS y est presente et facilite cette arrivee du point de vue legal, et par la prise en main de ces nouveaux venus.\nLa HIAS y est présente et facilite cette arrivée du point de vue légal, et par la prise en main de ces nouveaux venus.', 'Correct the errors in the following French text without telling me what you did: La majorités des accusés sont des femme, en grande partie pauvres, âgées de plus de et le plus souvent isolées.\nLa majorité des accusés sont des fem

# Training & Evaluation

In [ ]:
preds = []
refs = []

for i in range(50):
  input = dataset['test']['modified'][i][13:]
  ref = dataset['test']['sentence'][i]
  #print(input)
  out = gecchain.run(input)
  preds.append(out)
  refs.append(ref)

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate
def my_compute_metrics(preds, refs):
  # init metrics BLEU and ROUGE
  metric1 = evaluate.load('rouge')
  metric2 = evaluate.load('bleu')

  # compute metrics
  metrics = metric1.compute(predictions=preds, references=refs)
  metrics['bleu'] = metric2.compute(predictions=preds, references=refs)['bleu']

  return metrics

In [ ]:
# few shot
my_compute_metrics(preds,refs)

{'rouge1': 0.9334673565041935,
 'rouge2': 0.8914822608748006,
 'rougeL': 0.9231461107969804,
 'rougeLsum': 0.9207355059171958,
 'bleu': 0.8185264557198493}

In [ ]:
my_compute_metrics(preds,refs)

{'rouge1': 0.9233464469212085,
 'rouge2': 0.8769080473076418,
 'rougeL': 0.9150455514818161,
 'rougeLsum': 0.9115478586226633,
 'bleu': 0.7833426303955915}

Save files

In [ ]:
with open(r'/content/preds.txt', 'w') as fp:
  for item in preds:
    # write each item on a new line
    fp.write("%s\n" % item)

In [ ]:
with open(r'/content/refs.txt', 'w') as fp:
  for item in refs:
    # write each item on a new line
    fp.write("%s\n" % item)

In [ ]:
src = []
for i in range(50):
  input = dataset['test']['modified'][i][13:]
  src.append(input)

In [ ]:
with open(r'/content/src.txt', 'w') as fp:
  for item in src:
    # write each item on a new line
    fp.write("%s\n" % item)